In [ ]:
%%capture
from google.colab import drive
drive.mount('/content/drive', force_remount=True)
import pandas as pd
import json
!pip install demjson3
from demjson3 import decode
!pip install --upgrade transformers==4.2
from transformers import AutoTokenizer, AutoModel

In [ ]:
!pip freeze

absl-py==1.4.0
accelerate==1.5.2
aiohappyeyeballs==2.6.1
aiohttp==3.11.14
aiosignal==1.3.2
alabaster==1.0.0
albucore==0.0.23
albumentations==2.0.5
ale-py==0.10.2
altair==5.5.0
annotated-types==0.7.0
anyio==4.9.0
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
array_record==0.7.1
arviz==0.21.0
astropy==7.0.1
astropy-iers-data==0.2025.3.17.0.34.53
astunparse==1.6.3
atpublic==5.1
attrs==25.3.0
audioread==3.0.1
autograd==1.7.0
babel==2.17.0
backcall==0.2.0
beautifulsoup4==4.13.3
betterproto==2.0.0b6
bigframes==1.41.0
bigquery-magics==0.8.0
bleach==6.2.0
blinker==1.9.0
blis==1.2.0
blosc2==3.2.0
bokeh==3.6.3
Bottleneck==1.4.2
bqplot==0.12.44
branca==0.8.1
CacheControl==0.14.2
cachetools==5.5.2
catalogue==2.0.10
certifi==2025.1.31
cffi==1.17.1
chardet==5.2.0
charset-normalizer==3.4.1
chex==0.1.89
clarabel==0.10.0
click==8.1.8
cloudpathlib==0.21.0
cloudpickle==3.1.1
cmake==3.31.6
cmdstanpy==1.2.5
colorcet==3.1.0
colorlover==0.3.0
colour==0.1.5
community==1.0.0b1
confection==0.1.5
cons==0.4.6


In [ ]:
# tokenizer = AutoTokenizer.from_pretrained('allenai/specter')
# model = AutoModel.from_pretrained('allenai/specter')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [ ]:
# crashes here: "Ran out of available RAM" ...
# https://forum.micropython.org/viewtopic.php?t=9734
# https://www.geeksforgeeks.org/releasing-memory-in-python/ - THIS TO KNOW HOW MUCH MEMORY gc.collect() released
# https://github.com/tuhinkarmakar3882/RamCleaningUtility/blob/master/Ram%20Clearner%202.0.p
# IF IT STILL SOMEHOW STOPS, THEN JUST SAVE INTERMEDIARY DATA TO CSV


# emb_df = pd.DataFrame([], columns=[])
# import gc
# import ctypes
# import numpy

# def get_title_embeddings(row):
#   inputs = tokenizer([row["title"]], padding=True, truncation=True, return_tensors="pt", max_length=512)
#   result = model(**inputs)
#   temp_embeddings = result.last_hidden_state[:, 0, :]
#   del inputs
#   del result
#   gc.collect()
#   libc = ctypes.CDLL("libc.so.6")
#   libc.malloc_trim(0)
#   print( numpy.array(temp_embeddings.detach()) ) # GOOD CONVERSION TO NUMPY
#   # print( f'\{ "pmid":{row["pmid"]}, "title_embedding":{list(temp_embeddings)} \}')
#   # WRITING TO CSV ONLY HAPPENS AFTER THE FUNCTION COMPLETES, SO NO
#   # emb_df["title_embeddings"] = emb_df.append({"title_embeddings": temp_embeddings, "pmid":row["pmid"]}, ignore_index=True)
#   # emb_df.to_csv("/content/drive/MyDrive/emb_df.csv", index=False)
#   return temp_embeddings

# useless_output = df.apply( get_title_embeddings, axis=1 )

In [ ]:
# !pip3 install nltk
# !pip3 install gensim

import pandas as pd
from nltk.tokenize import word_tokenize
from gensim.models import Word2Vec
import gensim.downloader as api
import nltk
from nltk.corpus import stopwords
nltk.download('punkt_tab')
emb_df = pd.DataFrame([], columns=[])

from google.colab import drive
drive.mount('/content/drive', force_remount=True)
df = pd.read_csv("/content/drive/MyDrive/checkpoint_12_df.csv")
df = df.astype(str)

nltk.download('stopwords')
stopwords = set(stopwords.words('english'))

# Assuming df is your DataFrame with a column named 'title'
# titles = df['title'].tolist()
# df = pd.read_csv("/content/drive/MyDrive/checkpoint_12_df.csv")
# df = df.astype(str)

# fasttext-wiki-news-subwords-300
# conceptnet-numberbatch-17-06-300
# word2vec-ruscorpora-300
# word2vec-google-news-300
# glove-wiki-gigaword-50
# glove-wiki-gigaword-100
# glove-wiki-gigaword-200
# glove-wiki-gigaword-300
# glove-twitter-25
# glove-twitter-50
# glove-twitter-100
# glove-twitter-200
# __testing_word2vec-matrix-synopsis

model = api.load("glove-twitter-25")  # download the model and return as object ready for use
def get_column_embeddings(row):
  global model
  global column
  tokens = word_tokenize(row[column].lower())
  for token in tokens:
      if token in stopwords:
          tokens.remove(token)
  embeddings = [model[token] for token in tokens if token in model]
  if embeddings:
      return sum(embeddings) / len(embeddings)
  else:
      return [0] * model.vector_size

def get_title_abstract_embeddings(row):
  global model
  global column
  tokens = word_tokenize(row["title"].lower())
  tokens.extend( word_tokenize(row["abstract"].lower()) )
  for token in tokens:
      if token in stopwords:
          tokens.remove(token)
  embeddings = [model[token] for token in tokens if token in model]
  if embeddings:
      return sum(embeddings) / len(embeddings)
  else:
      return [0] * model.vector_size

column = "title"
emb_df["title_embeddings"] = df.apply( get_column_embeddings, axis=1 )
column = "abstract"
emb_df["abstract_embeddings"] = df.apply( get_column_embeddings, axis=1 )
emb_df["title_abstract_embeddings"] = df.apply( get_title_abstract_embeddings, axis=1 )
column = "keywords"
emb_df["keywords_embeddings"] = df.apply( get_column_embeddings, axis=1 )
column = "mesh"
emb_df["mesh_embeddings"] = df.apply( get_column_embeddings, axis=1 )
emb_df["year"] = df["year"]
emb_df.to_csv("/content/drive/MyDrive/emb_df.csv", index=False)

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


Mounted at /content/drive


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
check = pd.read_csv("/content/drive/MyDrive/emb_df.csv")
check = check.astype(str)
check.describe()

,title_embeddings,abstract_embeddings,title_abstract_embeddings
count,26806,26806,26806
unique,26526,26796,26799
top,[ 0.69586 -1.1469 -0.41797 -0.022311 -0.02...,[ 0.13151678 0.19295415 -0.4040065 0.042767...,[-0.00363883 0.08008853 -0.41465718 0.009823...
freq,44,2,2


In [ ]:
df.columns

Index(['abstract', 'authors', 'chemicals', 'citation', 'doi', 'history',
       'issn', 'issue', 'journal', 'keywords', 'mesh', 'pmc', 'pmid',
       'publication_types', 'title', 'url', 'volume', 'volume_issue', 'xml',
       'year', 'author_first', 'references', 'issn_data'],
      dtype='object')